# <center>Agent DQN doszkalany ewolucyjnie</center>
<center><img src="https://media.giphy.com/media/dyde6O8yn4oRh7R1Vk/source.gif" width="460" height="280" /></center>

*<center>Wykonali</center>*
*<center>Jakub Gros, Sebastian Kucharzyk</center>*


## Cel i Analiza Projektu
<p style='text-align: justify;'>
Jako cel obraliśmy sobie stworzenie maszynowego odpowiednika człowieka grającego w grę 
komputerową(tzw. agenta), który ucząc się na własnych błędach, będzie osiągał w niej coraz to lepsze 
rezultaty. Do tego celu nadaje się paradygmat procesu uczenia nazywany uczeniem ze wzmocnieniem,
polegający na umieszczeniu uczącego się agenta w nieznanym mu dotąd środowisku i pozwoleniu na 
wykonywanie dowolnych interakcji z otoczeniem. Agent w trakcie eksploracji doświadcza wielu różnych 
sytuacji zwanych stanami. Znajdując się w danym stanie wybiera, którą z obecnie możliwych akcji 
wykona, za co  finalnie otrzymuje pewną nagrodę lub karę. Wszystkie informacje na temat ruchów i 
nagród przechowuje w pamięci, aby nastepnie na ich podstawie móc oceniać, jaka akcja da mu 
największą nagrodę. Pozornie wydawałoby się, że rozwiązanie to nadaje się niemal idealnie do sytuacji 
wymagających, aby agent sam zebrał dane, na których następnie będzie się uczył, ale niestety posiada ono 
jedną znaczącą wadę - wraz z upływem czasu ilość danych w pamięci agenta rośnie, co z kolei powoduje 
spowolnienie podejmowania decyzji, a finalnie może nawet prowadzić do zapełnienia pamięci 
fizycznej urządzenia.
</p>


<p style='text-align: justify;'>
Aby uniknąć problemów związanych z nadmiarem danych, zdecydowaliśmny się na wykorzystanie innego paradygmatu, 
który pozwala zgeneralizować zbiór danych zamiast nieskończenie go poszerzać i przeszukiwać. Dokładniej 
rzecz biorąc, wybór padł na zastosowanie głębokiego Q-learningu, czyli połączenia uczenia 
ze wzmocnieniem z uczeniem głębokim. Polega on na zmniejszeniu nieograniczonej pamięci agenta 
do relatywnie małego bufora pamiętającego N ostatnich wyborów, który po każdej akcji wykonannej 
przez agenta jest wykorzystywany do  iteracyjnego dotrenowywania sieci neuronowej. Wykonuje się to za 
pomocą Q-wartości, czyli pewnej funkcji  <b>Q(S, A)</b> szacującej jak bardzo opłąca się podjąć akcję A 
będąc w stanie S. Cały powyższy proces prowadzi do stworzenia modelu Deep Q-Network(DQN).
</p>


<p style='text-align: justify;'>
Niemniej jednak nie chcieliśmy ograniczać się jedynie do uczenia głębokiego, więc postanowiliśmy
znaleźć zastosowanie dla innej dziedziny z zakresu stucznej inteligencji - algorytmów ewolucyjnych. 
Patrząc na sposób działania modelu DQN można spostrzec trzy obszary, które potencjalnie nadają się 
do ulepszenia ewolucyjnego:
</p>

1. Kształt sieci neuronowej
2. Wagi połączeń międzyneuronowych
3. Dane przechowywane w pamięci agentów

<p style='text-align: justify;'>
Po dokładnym przeanalizowaniu powyższych możliwości zdecydowaliśmy o odrzuceniu opcji numer 1 oraz 3. 
O wyeliminowaniu manipulacji kształtem sieci neuronowej zadecydował fakt, że ... . Natomiast jeśli 
chodzi o trzecią opcję, to jej odrzucenie wynika z natury DQN, której agent przechowuje w pamięci 
jedynie mały  fragment z całego zbioru danych, na którym był uczony, więc używanie go jako podstawę
do dalszego ulepszania najprawdopodobniej nie porwadziłoby do uzyskania wymiernych korzyści. 
W taki sposób stwierdziliśmy, że najciekawszym i(być może) najefektywniejszym podejściem będzie 
zastosowanie algorytmu genetycznego operującego na wagach połączeń międzyneuronowych.
</p>


## Ograniczenia sprzętowe
<p style='text-align: justify;'>
W momencie gdy zaimplementowaliśmy prostego i stosunkowo uniwersalnego agenta, który mógł być 
trenowany z wykorzystaniem większości środowisk dostępnych poprzez interfejs AI Gym, okazało się, 
że nasz sprzęt nie jest wystarczająco wydajny, aby zrealizować ten projekt w akceptowalnych ramach 
czasowych. Mówiąc konkretniej, trenowanie naszego modelu do stopnia, w którym jego wyniki są 
powtarzalne i można zacząć wyciągać z nich wnioski, w najprostszym środowisku(CartPole) zajmuje 
około 10 minut, natomiast w drugim co do złożoności LunarLander czas ten wzrasta aż do 45 minut. 
Po oszacowaniu, że do zakończenia projektu będziemy musieli trenować model w różnych konfiguracjach 
blisko 100 razy, liczba 75 godzin spędzona jednynie na trenowaniu LunarLandera byłaby zbyt duża.
</p>

## Zastosowane technologie
<p style='text-align: justify;'>
Kod projektu napisaliśmy w języku Python w wersji 3.6, z którego znakomita większość wykonuje się 
na procesorze. Karta graficzna jest odpowiedzialna za przeprowadzanie operacji czerpiących wymierne 
korzyści z wielowątkowości, a więc za trenowanie sieci neuronowej. Sama sieć oraz wszelkie operacje 
z nią związane pochodzą z wysokopoziomowego API Kerasa bazującego na bibliotece TensorFlow. Jeśli 
zaś chodzi o środowisko, w którym odbywa się trenowanie, to jest to CartPole z biblioteki AI Gym.
</p>




Celem projektu jest stworzenie i wytrenowanie agenta DQN w taki sposób, aby osiągał jak najlepsze rezultaty w grze
typu Cart Pole.


## Etapy działania

Projekt można podzielić na następujące fazy:
1. Implementacja naiwnego agenta zawsze podejmującego decyzję w sposób losowy
2. Implementacja agenta wykorzystującego prostą sieć neuronową
3. Poprawienie rezultatów osiąganych przez agenta poprzez zmiany w jego implementacji
4. Zebranie pamięci z pięciu agentów, a następnie ewolucyjne mieszanie ich parametrów tak,
aby otrzymać agenta z najlepszą bazą przypadków do nauki


## Model losowy

In [ ]:
import random
import gym
import tensorflow as tf
a = random.randint()


In [ ]:
NUM_OF_EPISODES = 200
FRAMES_PER_EPISODE = 1000
BATCH_SIZE = 32
GAME_ID = 'CartPole-v1'
CPU_ID = '/device:CPU:0'
print(0)


### Header

### Header

#### Header

##### Header

In [ ]:
import numpy